In [1]:
import pandas as pd
import numpy as np
import json
import time
import string
import re
from collections import Counter

In [2]:
df = pd.read_csv('ENG_scrambled_.csv')

In [3]:
df.columns

Index(['conversationId ', 'fromEmailAddress ', 'toRecipients ', 'subject ',
       'body.content ', 'receivedDateTime ', 'ccRecipients ', 'id ',
       'commentBy', 'comment', '_allow_permissions', 'type', 'from_user',
       'recipient_user'],
      dtype='object')

In [4]:
df_ = pd.read_csv('timob_index.csv')

In [5]:
len(df), len(df_)

(11616, 11616)

In [6]:
df_.columns

Index(['Issue.id', 'Issue.key', 'Summary', 'Issue.Type', 'Status',
       'Project.key', 'Project.name', 'Project.type', 'Project.lead',
       'Resolution', 'Assignee', 'Reporter', 'Creator', 'Created', 'Updated',
       'Resolved', 'Component.s', 'Labels', 'Description', 'Environment',
       'Custom.field..Tester.', 'Custom.field..gitBranch.', 'record_type',
       'commentDate', 'commentBy', 'comment'],
      dtype='object')

In [7]:
df['commentDate'] = df_['commentDate'].copy()

In [8]:
max(df['receivedDateTime '])

'2021-04-27T00:00:00'

In [9]:
import dateutil.parser as parser
from dateutil.relativedelta import relativedelta
import datetime

def date_clean(df):
    if pd.isnull(df) == False:
        try:
            date = parser.parse(df)
#             new_date = date + pd.DateOffset(years=7, months=4)
            date = date.strftime("%Y-%m-%dT%H:%M:%SZ")

            return date
        except Exception as e:
            return None
    else:
        return df
df['commentDate'] = df['commentDate'].apply(date_clean)

In [10]:
df['receivedDateTime '] = df['receivedDateTime '].apply(date_clean)

In [11]:
for i in range(len(df)):
    if pd.isnull(df['receivedDateTime '].loc[i]) == True or pd.isnull(df['commentDate'].loc[i])==True:
        continue
    date = parser.parse(df['receivedDateTime '].loc[i])
    cmd = parser.parse(df['commentDate'].loc[i])
    diff = date-cmd
    cmd = cmd + pd.DateOffset(days = diff.days+1)
    df['commentDate'].loc[i] = cmd.strftime("%Y-%m-%dT%H:%M:%SZ")
    

C:\Users\manoj praveen\Anaconda3\lib\site-packages\pandas\core\indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [12]:
max([w for w in df['commentDate'] if pd.isnull(w)==False])
# datetime.datetime.strptime("2013-10-05T01:21:07Z", "%Y-%m-%dT%H:%M:%SZ")

'2021-04-27T10:41:00Z'

In [13]:
with open("final_fake_transient.json", 'r') as f:
    final_fake_tr = json.loads(f.read())

In [14]:
ex_emp = [w for w in final_fake_tr.keys() if w.endswith("@example.com")]

In [15]:
fake_id_df = pd.read_csv('fake_users_with_ids.csv')
def user_tag(df):
    return 'USER_' + df
fake_ids = fake_id_df[['emailId', 'userId']].copy()
fake_ids['userId'] = fake_ids['userId'].apply(user_tag)
fake_tr = dict(zip(list(fake_ids['emailId']), list(fake_ids['userId'])))

In [16]:
with open('ENG_cog_dict.json', 'r') as f:
    cog_dict = json.loads(f.read())

In [17]:
cog_dict.update(fake_tr)

In [18]:
def wrangle_recp():
    resp = []
    for i in range(len(df)):
        ccs = [w for w in eval(df['ccRecipients '].loc[i]) if w not in eval(df['fromEmailAddress '].loc[i])[0]]
        try:
            resp.append(eval(df['toRecipients '].loc[i]) + ccs)
        except Exception as e:
            print(i)
            pass
    return resp

In [19]:
def spe_recp():
    resp = []
    for i in range(len(df)):
        to = [w for w in eval(df['_allow_permissions'].loc[i]) if w!= eval(df['commentBy'].loc[i])[0]]
        resp.append(to)
    return resp

In [20]:
df['commentDate']

0                         NaN
1        2018-08-14T18:27:00Z
2        2018-08-14T18:16:00Z
3        2018-08-14T21:37:00Z
4        2018-08-14T07:17:00Z
                 ...         
11611    2021-04-27T10:41:00Z
11612    2021-04-27T03:14:00Z
11613                     NaN
11614    2021-04-27T07:56:00Z
11615    2021-04-27T06:31:00Z
Name: commentDate, Length: 11616, dtype: object

In [21]:
def user_dict(df):
    res = {}
    user_id = []
    user_display_name = []
    email = []

    for i in df:
        user_id.append(cog_dict[i])
        user_display_name.append(final_fake_tr[i])
        email.append(i)
    res['user_id'] = user_id
    res['user_display_name'] = user_display_name
#     res['email'] = email
    res_df = pd.DataFrame.from_dict(res)
    return json.loads(json.dumps(res_df.to_dict('records')))

In [22]:
def evaluator(df):
    return eval(df)

In [23]:
df['fromEmailAddress '] = df['fromEmailAddress '].apply(lambda x: eval(x))

In [24]:
df['toRecipients '] = df['toRecipients '].apply(lambda x: eval(x))
df['ccRecipients '] = df['ccRecipients '].apply(lambda x: eval(x))

In [25]:
df.head()

,conversationId,fromEmailAddress,toRecipients,subject,body.content,receivedDateTime,ccRecipients,id,commentBy,comment,_allow_permissions,type,from_user,recipient_user,commentDate
0,60016,[christopher.franke@gmail.com],[richard.ellis@example.com],iphone simulator: transportation path Figure,just thought i'd let you know that in titanium...,2018-08-14T00:00:00Z,"[liam.aylward@example.com, christopher.franke@...",AAMKAIDuuITaLzXdqAwOqBJFlsGnPEnljmaFKKKGGTerZR...,['liam.aylward@example.com'],NaN,"['manojpraveen445@gmail.com', 'christopher.fra...",email-message,Christopher Franke,Richard Ellis,NaN
1,60016,[christopher.franke@gmail.com],[richard.ellis@example.com],iphone simulator: transportation path Figure,just thought i'd let you know that in titanium...,2018-08-14T00:00:00Z,"[liam.aylward@example.com, christopher.franke@...",AAMKALJYLoGJQpGiRqXmqbjeuHAfrCnlMBlZBnfbAbonqc...,['christopher.franke@gmail.com'],i also get this in 1.6.0 with systematic varia...,"['manojpraveen445@gmail.com', 'christopher.fra...",email-message,Christopher Franke,Richard Ellis,2018-08-14T18:27:00Z
2,60016,[christopher.franke@gmail.com],[richard.ellis@example.com],iphone simulator: transportation path Figure,just thought i'd let you know that in titanium...,2018-08-14T00:00:00Z,"[liam.aylward@example.com, christopher.franke@...",AAMKApbePeTmSuCChwhQUENHHXNbetmzPEaiYANaDElQeL...,['laurent.beaudoin@gmail.com'],"michael,\n\nif you are willing to hack up your...","['manojpraveen445@gmail.com', 'christopher.fra...",email-message,Christopher Franke,Richard Ellis,2018-08-14T18:16:00Z
3,60016,[christopher.franke@gmail.com],[richard.ellis@example.com],iphone simulator: transportation path Figure,just thought i'd let you know that in titanium...,2018-08-14T00:00:00Z,"[liam.aylward@example.com, christopher.franke@...",AAMKADdThFgcKsogapXsgTaLemanknIISsXqsfIrRIJIGf...,['lu.shanglei@gmail.com'],i'm experiencing this with 2.1.3rc,"['manojpraveen445@gmail.com', 'christopher.fra...",email-message,Christopher Franke,Richard Ellis,2018-08-14T21:37:00Z
4,60016,[christopher.franke@gmail.com],[richard.ellis@example.com],iphone simulator: transportation path Figure,just thought i'd let you know that in titanium...,2018-08-14T00:00:00Z,"[liam.aylward@example.com, christopher.franke@...",AAMKAIVHcvnXXiVcjQxbKxpcJaatXnNQbbWwQeXkVjjzbP...,['rhonda.glenn@example.com'],pr pending personal://github.com/appcelerator/...,"['manojpraveen445@gmail.com', 'christopher.fra...",email-message,Christopher Franke,Richard Ellis,2018-08-14T07:17:00Z


In [26]:
user_dict(['richard.ellis@example.com'])

[{'user_id': 'USER_374b2ff7-020f-4571-959c-b0a3826702d7',
  'user_display_name': 'Richard Ellis'}]

In [27]:
import random
def gen_id(df):
    ps = ''.join(random.choice(string.ascii_letters) for x in range(8))
    return str(df) + ps

In [28]:
def gen_team_id():
    ps = ''.join(random.choices(string.ascii_lowercase + string.digits, k=8)) + '-' +''.join(random.choices(string.ascii_letters + string.digits, k=4)) + '-' + ''.join(random.choices(string.ascii_lowercase + string.digits, k=4)) + '-' + ''.join(random.choices(string.ascii_lowercase + string.digits, k=4)) + '-' + ''.join(random.choices(string.ascii_lowercase + string.digits, k=12))
    return ps

In [29]:
def gen_channel_id():
    ps = str(random.randint(10, 99)) + ':00' + ''.join(random.choices(string.ascii_lowercase + string.digits, k=43)) 
    return str(ps)

In [30]:
gen_channel_id()

'83:00pnquudbahowv2mntwt7ghzoege1gtqoshy8ic1z0t98'

In [31]:
gen_team_id()

'ovwmdltp-2oFC-wuzt-kku8-p5lpl80dvp3a'

In [32]:
gen_id(60016)

'60016IQTGwWWQ'

In [33]:
df_['Component.s'].fillna('nan', inplace=True)

In [34]:
df_['Component.s'].nunique()

15

In [35]:
df_['Component.s'].value_counts()

Android           5304
iOS               4554
CLI                456
BlackBerry         384
Tooling            234
MobileWeb          210
TiAPI              126
LiveView            90
nan                 90
Code Processor      78
Core                66
Release              6
Tizen                6
Specification        6
Drillbit             6
Name: Component.s, dtype: int64

In [36]:
df['teams'] = df_['Component.s'].copy()

In [37]:
team_names = ['Hexspeak', 'Concept Squad', 'Aztec Allure', 'Java Our Souls', 'Virtual Velocity', 
    'Theory Of Perfection', 'Byte Optimizers', 'Hack Frenzy', 'Team Inspiration', 
    'Ping Intelligence', 'Seek Syndrome', 'Optimal Circuits', 'Strategy Team', 'Crude Theory', 'Tizen']

In [38]:
len(team_names)

15

In [39]:
'nan' in df_['Component.s'].unique()

True

In [40]:
team_replace = dict(zip(list(df['teams'].unique()), team_names))

In [41]:
team_replace

{'iOS': 'Hexspeak',
 'Android': 'Concept Squad',
 'TiAPI': 'Aztec Allure',
 'Drillbit': 'Java Our Souls',
 'Tooling': 'Virtual Velocity',
 'nan': 'Theory Of Perfection',
 'Core': 'Byte Optimizers',
 'CLI': 'Hack Frenzy',
 'MobileWeb': 'Team Inspiration',
 'BlackBerry': 'Ping Intelligence',
 'Code Processor': 'Seek Syndrome',
 'Specification': 'Optimal Circuits',
 'LiveView': 'Strategy Team',
 'Release': 'Crude Theory',
 'Tizen': 'Tizen'}

In [42]:
# for i in team_replace:
#     print(type(i))

In [43]:
np.nan in team_replace

False

In [44]:
def faker_comps(df):
    if df in team_replace:
        for word, initial in team_replace.items():
            df = df.replace(word, initial)
        return df
    else:
        print(df)

    
df['teams'] = df['teams'].apply(faker_comps)

In [45]:
df.head()

,conversationId,fromEmailAddress,toRecipients,subject,body.content,receivedDateTime,ccRecipients,id,commentBy,comment,_allow_permissions,type,from_user,recipient_user,commentDate,teams
0,60016,[christopher.franke@gmail.com],[richard.ellis@example.com],iphone simulator: transportation path Figure,just thought i'd let you know that in titanium...,2018-08-14T00:00:00Z,"[liam.aylward@example.com, christopher.franke@...",AAMKAIDuuITaLzXdqAwOqBJFlsGnPEnljmaFKKKGGTerZR...,['liam.aylward@example.com'],NaN,"['manojpraveen445@gmail.com', 'christopher.fra...",email-message,Christopher Franke,Richard Ellis,NaN,Hexspeak
1,60016,[christopher.franke@gmail.com],[richard.ellis@example.com],iphone simulator: transportation path Figure,just thought i'd let you know that in titanium...,2018-08-14T00:00:00Z,"[liam.aylward@example.com, christopher.franke@...",AAMKALJYLoGJQpGiRqXmqbjeuHAfrCnlMBlZBnfbAbonqc...,['christopher.franke@gmail.com'],i also get this in 1.6.0 with systematic varia...,"['manojpraveen445@gmail.com', 'christopher.fra...",email-message,Christopher Franke,Richard Ellis,2018-08-14T18:27:00Z,Hexspeak
2,60016,[christopher.franke@gmail.com],[richard.ellis@example.com],iphone simulator: transportation path Figure,just thought i'd let you know that in titanium...,2018-08-14T00:00:00Z,"[liam.aylward@example.com, christopher.franke@...",AAMKApbePeTmSuCChwhQUENHHXNbetmzPEaiYANaDElQeL...,['laurent.beaudoin@gmail.com'],"michael,\n\nif you are willing to hack up your...","['manojpraveen445@gmail.com', 'christopher.fra...",email-message,Christopher Franke,Richard Ellis,2018-08-14T18:16:00Z,Hexspeak
3,60016,[christopher.franke@gmail.com],[richard.ellis@example.com],iphone simulator: transportation path Figure,just thought i'd let you know that in titanium...,2018-08-14T00:00:00Z,"[liam.aylward@example.com, christopher.franke@...",AAMKADdThFgcKsogapXsgTaLemanknIISsXqsfIrRIJIGf...,['lu.shanglei@gmail.com'],i'm experiencing this with 2.1.3rc,"['manojpraveen445@gmail.com', 'christopher.fra...",email-message,Christopher Franke,Richard Ellis,2018-08-14T21:37:00Z,Hexspeak
4,60016,[christopher.franke@gmail.com],[richard.ellis@example.com],iphone simulator: transportation path Figure,just thought i'd let you know that in titanium...,2018-08-14T00:00:00Z,"[liam.aylward@example.com, christopher.franke@...",AAMKAIVHcvnXXiVcjQxbKxpcJaatXnNQbbWwQeXkVjjzbP...,['rhonda.glenn@example.com'],pr pending personal://github.com/appcelerator/...,"['manojpraveen445@gmail.com', 'christopher.fra...",email-message,Christopher Franke,Richard Ellis,2018-08-14T07:17:00Z,Hexspeak


In [46]:
len('040c8e11-f6c0-45d9-bf15-524ac3e6cc0e')

36

In [47]:
team_ids = []
for i in range(len(team_names)):
    team_ids.append(gen_team_id())

In [48]:
team_tr = dict(zip(team_names, team_ids))

In [49]:
df['commentBy'] = df['commentBy'].apply(lambda x: eval(x)[0])

In [50]:
final_fake_tr[df['commentBy'].loc[2]]

'Laurent Beaudoin'

In [51]:
df['comment_user_name'] = df['commentBy'].apply(lambda x: final_fake_tr[x])

In [52]:
df.head()

,conversationId,fromEmailAddress,toRecipients,subject,body.content,receivedDateTime,ccRecipients,id,commentBy,comment,_allow_permissions,type,from_user,recipient_user,commentDate,teams,comment_user_name
0,60016,[christopher.franke@gmail.com],[richard.ellis@example.com],iphone simulator: transportation path Figure,just thought i'd let you know that in titanium...,2018-08-14T00:00:00Z,"[liam.aylward@example.com, christopher.franke@...",AAMKAIDuuITaLzXdqAwOqBJFlsGnPEnljmaFKKKGGTerZR...,liam.aylward@example.com,NaN,"['manojpraveen445@gmail.com', 'christopher.fra...",email-message,Christopher Franke,Richard Ellis,NaN,Hexspeak,Liam Aylward
1,60016,[christopher.franke@gmail.com],[richard.ellis@example.com],iphone simulator: transportation path Figure,just thought i'd let you know that in titanium...,2018-08-14T00:00:00Z,"[liam.aylward@example.com, christopher.franke@...",AAMKALJYLoGJQpGiRqXmqbjeuHAfrCnlMBlZBnfbAbonqc...,christopher.franke@gmail.com,i also get this in 1.6.0 with systematic varia...,"['manojpraveen445@gmail.com', 'christopher.fra...",email-message,Christopher Franke,Richard Ellis,2018-08-14T18:27:00Z,Hexspeak,Christopher Franke
2,60016,[christopher.franke@gmail.com],[richard.ellis@example.com],iphone simulator: transportation path Figure,just thought i'd let you know that in titanium...,2018-08-14T00:00:00Z,"[liam.aylward@example.com, christopher.franke@...",AAMKApbePeTmSuCChwhQUENHHXNbetmzPEaiYANaDElQeL...,laurent.beaudoin@gmail.com,"michael,\n\nif you are willing to hack up your...","['manojpraveen445@gmail.com', 'christopher.fra...",email-message,Christopher Franke,Richard Ellis,2018-08-14T18:16:00Z,Hexspeak,Laurent Beaudoin
3,60016,[christopher.franke@gmail.com],[richard.ellis@example.com],iphone simulator: transportation path Figure,just thought i'd let you know that in titanium...,2018-08-14T00:00:00Z,"[liam.aylward@example.com, christopher.franke@...",AAMKADdThFgcKsogapXsgTaLemanknIISsXqsfIrRIJIGf...,lu.shanglei@gmail.com,i'm experiencing this with 2.1.3rc,"['manojpraveen445@gmail.com', 'christopher.fra...",email-message,Christopher Franke,Richard Ellis,2018-08-14T21:37:00Z,Hexspeak,Lu Shanglei
4,60016,[christopher.franke@gmail.com],[richard.ellis@example.com],iphone simulator: transportation path Figure,just thought i'd let you know that in titanium...,2018-08-14T00:00:00Z,"[liam.aylward@example.com, christopher.franke@...",AAMKAIVHcvnXXiVcjQxbKxpcJaatXnNQbbWwQeXkVjjzbP...,rhonda.glenn@example.com,pr pending personal://github.com/appcelerator/...,"['manojpraveen445@gmail.com', 'christopher.fra...",email-message,Christopher Franke,Richard Ellis,2018-08-14T07:17:00Z,Hexspeak,Rhonda Glenn


In [53]:
team_tr

{'Hexspeak': 'dcdtozsa-r4dh-3buz-j7qt-onvge5qxco8f',
 'Concept Squad': 'tvlrau38-rJgb-ggv6-rzxc-v3ngebv0e00d',
 'Aztec Allure': 'zrw2282e-m1om-tbex-4f5h-av74y9x7xc97',
 'Java Our Souls': 'gt2z7xg8-ISx6-x56l-bmra-d68yo72bx2oc',
 'Virtual Velocity': 'yua78p14-8mDj-8b3c-cb9k-e9398atzng5b',
 'Theory Of Perfection': 'znasbg0s-oQtw-47pr-nsqm-iqpj6ejt6pe1',
 'Byte Optimizers': 'q6146ii8-0gFk-wk3s-xbvx-3pp88hhu6z1o',
 'Hack Frenzy': 'p8cgsq0o-vMxk-es3d-sgy1-qhysuqeesqzh',
 'Team Inspiration': '1tw8zm7z-tSGG-ivaj-hxp6-66xe3dl40p9a',
 'Ping Intelligence': 'xi8dhl7h-8tHr-ky2h-rcb5-kibmnquafo91',
 'Seek Syndrome': 'ptanf9p8-0bNw-e0yd-grnj-995odklxgu41',
 'Optimal Circuits': 'uisius4s-OIgU-6knp-3nyg-w3hx2dtqgbmy',
 'Strategy Team': 'c4hd92kx-xDBT-l5gz-7fei-gqfy4e1eybba',
 'Crude Theory': 'bkls2a7a-TCQx-x8bj-8n5h-544jhvwuda2a',
 'Tizen': 'kddls09u-7NG2-zuy1-jvaf-z5lkeea73bin'}

In [54]:
df[df['teams'] == 'Core']

,conversationId,fromEmailAddress,toRecipients,subject,body.content,receivedDateTime,ccRecipients,id,commentBy,comment,_allow_permissions,type,from_user,recipient_user,commentDate,teams,comment_user_name


In [55]:
df_['Component.s'].nunique(), len(team_tr)

(15, 15)

In [56]:
team_tr[df['teams'].loc[1]]

'dcdtozsa-r4dh-3buz-j7qt-onvge5qxco8f'

In [57]:
df.dropna(subset=['comment'], inplace=True)
df.reset_index(drop=True, inplace=True)

In [58]:
df['_allow_names'] = df['_allow_permissions'].apply(lambda x: [final_fake_tr[w] for w in eval(x)])

In [59]:
random.choices([1,2,3], k=random.randint(1,2))

[2, 1]

In [60]:
df['mentions'] = df['_allow_names'].apply(lambda x: random.choices(x, k=random.randint(1, len(list(set(x))))) if random.randint(0,1)==1 else '')

In [61]:
df['mentions'].loc[0]

['Laurent Beaudoin',
 'Lu Shanglei',
 'Manoj Praveen',
 'Christopher Franke',
 'Lu Shanglei']

In [62]:
df['teams'].unique()

array(['Hexspeak', 'Concept Squad', 'Aztec Allure', 'Java Our Souls',
       'Virtual Velocity', 'Theory Of Perfection', 'Byte Optimizers',
       'Hack Frenzy', 'Team Inspiration', 'Ping Intelligence',
       'Seek Syndrome', 'Optimal Circuits', 'Strategy Team',
       'Crude Theory', 'Tizen'], dtype=object)

In [63]:
df['team_id'] = df['teams'].apply(lambda x: team_tr[x] if pd.isnull(x)==False else None)

In [64]:
for i in range(len(df)):
    mentionString = ''
    for j in range(len(df['mentions'].loc[i])):
        if df['mentions'].loc[i][j]!= '':
            mentionString = mentionString + '@' + df['mentions'].loc[i][j] + ' '
    df['comment'].loc[i] = df['comment'].loc[i] + mentionString

C:\Users\manoj praveen\Anaconda3\lib\site-packages\pandas\core\indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [65]:
len(df)

9677

In [66]:
df['comment'].loc[9561]

"i tried it in a module which works. please see this line.\n\npersonal://github.com/wappzapp/html5video-appcelerator-module/considered/master/src/tv/wappzapp/html5video/webvideoproxy.java#l9pages\n\nshouldn't be that hard of a fix. i like the way you guys do it way better.@Mercurybeatz "

In [67]:
df['_allow_names'].loc[9561], df['mentions'].loc[9561]

(['Daiju Takase',
  'Alan Reid',
  'Dory Chamoun',
  'Mansour Bin',
  'Lee Binding',
  'Mercurybeatz',
  'Timothy Price'],
 ['Mercurybeatz'])

In [68]:
def user_dict_c(df):
    res = {}
    user_id = []
    user_display_name = []
    email = []
    user_id.append(cog_dict[df])
    user_display_name.append(final_fake_tr[df])
    email.append(df)
    res['user_id'] = user_id
    res['user_display_name'] = user_display_name
#     res['email'] = email
    res_df = pd.DataFrame.from_dict(res)
    return json.loads(json.dumps(res_df.to_dict('records')))

In [69]:
df['from_'] = df['commentBy'].apply(user_dict_c)

In [70]:
df['from_'] = df['from_'].apply(lambda x: x[0])

In [71]:
channel_names = ['General', 'Engineering', 'ML', 'Reviews', 'Training', 'Testing']
channel_ids = []
for i in range(len(channel_names)):
    channel_ids.append(gen_channel_id())
channel_tr = dict(zip(channel_names, channel_ids))
def create_channels(df):
    return random.choice(channel_names)    

In [72]:
df['channel_name'] = df['teams'].apply(create_channels)

In [73]:
dc = df[['conversationId ', 'channel_name']].groupby(by='conversationId ').max()

In [74]:
dc.reset_index(inplace=True)

In [75]:
df.drop(columns=['channel_name'], inplace=True)

In [76]:
df = df.merge(dc, on='conversationId ')

In [77]:
channel_tr

{'General': '10:00nqgsj93lydh8uvfad9u3eqpqtca9gaq6nha8kyzcogn',
 'Engineering': '18:00ulilx1ire2dor4j0kyhppy9qfjlr5qfzny130w6c76r',
 'ML': '81:00dcmfpetdai6ylews2c47ob9bbefuxw4e31ai0lzm3gl',
 'Reviews': '82:005bhr0trmw478pt10y7ty588bt11gh7laogcr4bma613',
 'Training': '37:0021crw9eeo2v477n5bcyhdkf9zemolgz63z4jck0ydb1',
 'Testing': '84:00rqnz0p5slnguvv95pgt4rvfjy6c34uqxqiwagz19lyk'}

In [78]:
df['channel_id'] = df['channel_name'].apply(lambda x: channel_tr[x])

In [79]:
df.head()

,conversationId,fromEmailAddress,toRecipients,subject,body.content,receivedDateTime,ccRecipients,id,commentBy,comment,...,recipient_user,commentDate,teams,comment_user_name,_allow_names,mentions,team_id,from_,channel_name,channel_id
0,60016,[christopher.franke@gmail.com],[richard.ellis@example.com],iphone simulator: transportation path Figure,just thought i'd let you know that in titanium...,2018-08-14T00:00:00Z,"[liam.aylward@example.com, christopher.franke@...",AAMKALJYLoGJQpGiRqXmqbjeuHAfrCnlMBlZBnfbAbonqc...,christopher.franke@gmail.com,i also get this in 1.6.0 with systematic varia...,...,Richard Ellis,2018-08-14T18:27:00Z,Hexspeak,Christopher Franke,"[Manoj Praveen, Christopher Franke, Richard El...","[Laurent Beaudoin, Lu Shanglei, Manoj Praveen,...",dcdtozsa-r4dh-3buz-j7qt-onvge5qxco8f,"{'user_id': 'christopher.franke@gmail.com', 'u...",Training,37:0021crw9eeo2v477n5bcyhdkf9zemolgz63z4jck0ydb1
1,60016,[christopher.franke@gmail.com],[richard.ellis@example.com],iphone simulator: transportation path Figure,just thought i'd let you know that in titanium...,2018-08-14T00:00:00Z,"[liam.aylward@example.com, christopher.franke@...",AAMKApbePeTmSuCChwhQUENHHXNbetmzPEaiYANaDElQeL...,laurent.beaudoin@gmail.com,"michael,\n\nif you are willing to hack up your...",...,Richard Ellis,2018-08-14T18:16:00Z,Hexspeak,Laurent Beaudoin,"[Manoj Praveen, Christopher Franke, Richard El...",,dcdtozsa-r4dh-3buz-j7qt-onvge5qxco8f,"{'user_id': 'laurent.beaudoin@gmail.com', 'use...",Training,37:0021crw9eeo2v477n5bcyhdkf9zemolgz63z4jck0ydb1
2,60016,[christopher.franke@gmail.com],[richard.ellis@example.com],iphone simulator: transportation path Figure,just thought i'd let you know that in titanium...,2018-08-14T00:00:00Z,"[liam.aylward@example.com, christopher.franke@...",AAMKADdThFgcKsogapXsgTaLemanknIISsXqsfIrRIJIGf...,lu.shanglei@gmail.com,i'm experiencing this with 2.1.3rc,...,Richard Ellis,2018-08-14T21:37:00Z,Hexspeak,Lu Shanglei,"[Manoj Praveen, Christopher Franke, Richard El...",,dcdtozsa-r4dh-3buz-j7qt-onvge5qxco8f,"{'user_id': 'lu.shanglei@gmail.com', 'user_dis...",Training,37:0021crw9eeo2v477n5bcyhdkf9zemolgz63z4jck0ydb1
3,60016,[christopher.franke@gmail.com],[richard.ellis@example.com],iphone simulator: transportation path Figure,just thought i'd let you know that in titanium...,2018-08-14T00:00:00Z,"[liam.aylward@example.com, christopher.franke@...",AAMKAIVHcvnXXiVcjQxbKxpcJaatXnNQbbWwQeXkVjjzbP...,rhonda.glenn@example.com,pr pending personal://github.com/appcelerator/...,...,Richard Ellis,2018-08-14T07:17:00Z,Hexspeak,Rhonda Glenn,"[Manoj Praveen, Christopher Franke, Richard El...",[Lu Shanglei],dcdtozsa-r4dh-3buz-j7qt-onvge5qxco8f,{'user_id': 'USER_8dd3acad-f6aa-4f69-808b-a35a...,Training,37:0021crw9eeo2v477n5bcyhdkf9zemolgz63z4jck0ydb1
4,60016,[christopher.franke@gmail.com],[richard.ellis@example.com],iphone simulator: transportation path Figure,just thought i'd let you know that in titanium...,2018-08-14T00:00:00Z,"[liam.aylward@example.com, christopher.franke@...",AAMKACGbgoQFaeYHAANbDTLbkAoEqOsLbnjedXBYCHprEm...,manojpraveen445@gmail.com,http://technologies.appcelerator.com/question/...,...,Richard Ellis,2018-08-14T07:15:00Z,Hexspeak,Manoj Praveen,"[Manoj Praveen, Christopher Franke, Richard El...",,dcdtozsa-r4dh-3buz-j7qt-onvge5qxco8f,{'user_id': 'USER_1f4c0330-32c9-49dd-8e25-4fc6...,Training,37:0021crw9eeo2v477n5bcyhdkf9zemolgz63z4jck0ydb1


In [80]:
main_df = df.drop_duplicates(subset='conversationId ',keep='first')

In [81]:
user_dict_c(main_df['commentBy'].loc[0])

[{'user_id': 'christopher.franke@gmail.com',
  'user_display_name': 'Christopher Franke'}]

In [82]:
main_df.columns

Index(['conversationId ', 'fromEmailAddress ', 'toRecipients ', 'subject ',
       'body.content ', 'receivedDateTime ', 'ccRecipients ', 'id ',
       'commentBy', 'comment', '_allow_permissions', 'type', 'from_user',
       'recipient_user', 'commentDate', 'teams', 'comment_user_name',
       '_allow_names', 'mentions', 'team_id', 'from_', 'channel_name',
       'channel_id'],
      dtype='object')

In [83]:
main_df.head()

,conversationId,fromEmailAddress,toRecipients,subject,body.content,receivedDateTime,ccRecipients,id,commentBy,comment,...,recipient_user,commentDate,teams,comment_user_name,_allow_names,mentions,team_id,from_,channel_name,channel_id
0,60016,[christopher.franke@gmail.com],[richard.ellis@example.com],iphone simulator: transportation path Figure,just thought i'd let you know that in titanium...,2018-08-14T00:00:00Z,"[liam.aylward@example.com, christopher.franke@...",AAMKALJYLoGJQpGiRqXmqbjeuHAfrCnlMBlZBnfbAbonqc...,christopher.franke@gmail.com,i also get this in 1.6.0 with systematic varia...,...,Richard Ellis,2018-08-14T18:27:00Z,Hexspeak,Christopher Franke,"[Manoj Praveen, Christopher Franke, Richard El...","[Laurent Beaudoin, Lu Shanglei, Manoj Praveen,...",dcdtozsa-r4dh-3buz-j7qt-onvge5qxco8f,"{'user_id': 'christopher.franke@gmail.com', 'u...",Training,37:0021crw9eeo2v477n5bcyhdkf9zemolgz63z4jck0ydb1
5,60648,[huey.dunbar@example.com],[tommy.ivo@example.com],via tender: newby - error when launching nodes...,"assigned to <a href=\n""http://support.appceler...",2018-08-14T00:00:00Z,"[tommy.ivo@example.com, tommy.ivo@example.com,...",AAMKAzgVFaUsZiloXsOlouOwjckesQDriUijOXuZRvwUdo...,tommy.ivo@example.com,i've attached a replacement prereq.py that sho...,...,Tommy Ivo,2018-08-14T19:22:00Z,Concept Squad,Tommy Ivo,"[Cyndy Violette, Simon Cawkwell, Huey Dunbar, ...",,tvlrau38-rJgb-ggv6-rzxc-v3ngebv0e00d,{'user_id': 'USER_96ec5a12-8d0d-4c7f-b62c-2159...,Training,37:0021crw9eeo2v477n5bcyhdkf9zemolgz63z4jck0ydb1
10,60659,[robert.langdon@example.com],[peter.st@example.com],services - reseexampleh connect integration,description\nthe purpose of this services is t...,2018-08-14T00:00:00Z,"[huey.dunbar@example.com, huey.dunbar@example....",AAMKAYIJtPMTNywhyyXdWPCcTWjPPClYHaDonwloarWLnM...,huey.dunbar@example.com,execute services call\nyou can execute a resee...,...,Peter St,2018-08-14T19:22:00Z,Hexspeak,Huey Dunbar,"[Robert Langdon, Peter St, Jerry Hunter, Huey ...",,dcdtozsa-r4dh-3buz-j7qt-onvge5qxco8f,{'user_id': 'USER_b570965f-414c-4868-9dc2-ae2b...,Training,37:0021crw9eeo2v477n5bcyhdkf9zemolgz63z4jck0ydb1
15,60677,[peter.miller@gmail.com],[oliver.wilson@example.com],"navbar color on ""more"" tab",my tiapp.xml file contains about 7 windows. al...,2018-08-14T00:00:00Z,"[peter.st@example.com, huey.dunbar@example.com...",AAMKAIidqMCjMLkQBcsfHCOjOOKIqSOwXJhBJBDqMUtVcP...,peter.st@example.com,"okay, possible proposal for implementation fun...",...,Oliver Wilson,2018-08-14T19:22:00Z,Hexspeak,Peter St,"[Oliver Wilson, Timothy Price, Huey Dunbar, Pe...","[Peter St, Oliver Wilson]",dcdtozsa-r4dh-3buz-j7qt-onvge5qxco8f,{'user_id': 'USER_2f46bee1-e642-4850-a307-a4bd...,Training,37:0021crw9eeo2v477n5bcyhdkf9zemolgz63z4jck0ydb1
20,60757,[denis.ducharme@gmail.com],[oliver.wilson@example.com],notationss are too limited,i've been tinkering around with ti to see if i...,2018-08-14T00:00:00Z,"[al.franken@example.com, peter.st@example.com,...",AAMKArAFMDpNrkxnVzCeSVqmLjoRhIIfnoPKizcEDVufsu...,al.franken@example.com,another@Tommy Ivo @Daniel Kopans @Denis Duchar...,...,Oliver Wilson,2018-08-14T19:23:00Z,Hexspeak,Al Franken,"[Daniel Kopans, Tommy Ivo, Oliver Wilson, Al F...","[Tommy Ivo, Daniel Kopans, Denis Ducharme, Den...",dcdtozsa-r4dh-3buz-j7qt-onvge5qxco8f,{'user_id': 'USER_4cdfaeb3-adf5-4717-bd47-2024...,Training,37:0021crw9eeo2v477n5bcyhdkf9zemolgz63z4jck0ydb1


In [84]:
main_df = main_df[['_allow_permissions', 'id ', 'conversationId ', 'receivedDateTime ', 
                   'conversationId ', 'id ', 'team_id', 'teams', 'channel_id',
       'channel_name', 'body.content ', 'fromEmailAddress ', 'receivedDateTime ', 
          'type' ]].copy()

In [85]:
main_df.columns = ['_allow_permissions', 'id', 'conversation_id', 'date', 
                   'conversation_id', 'message_id', 'team_id', 'team_name', 'channel_id',
       'channel_name', 'body', 'from', 'date', 
          'type' ]

In [86]:
cog_dict['christopher.franke@gmail.com']

'christopher.franke@gmail.com'

In [87]:
main_df.head()

,_allow_permissions,id,conversation_id,date,conversation_id,message_id,team_id,team_name,channel_id,channel_name,body,from,date,type
0,"['manojpraveen445@gmail.com', 'christopher.fra...",AAMKALJYLoGJQpGiRqXmqbjeuHAfrCnlMBlZBnfbAbonqc...,60016,2018-08-14T00:00:00Z,60016,AAMKALJYLoGJQpGiRqXmqbjeuHAfrCnlMBlZBnfbAbonqc...,dcdtozsa-r4dh-3buz-j7qt-onvge5qxco8f,Hexspeak,37:0021crw9eeo2v477n5bcyhdkf9zemolgz63z4jck0ydb1,Training,just thought i'd let you know that in titanium...,[christopher.franke@gmail.com],2018-08-14T00:00:00Z,email-message
5,"['cyndy.violette@example.com', 'simon.cawkwell...",AAMKAzgVFaUsZiloXsOlouOwjckesQDriUijOXuZRvwUdo...,60648,2018-08-14T00:00:00Z,60648,AAMKAzgVFaUsZiloXsOlouOwjckesQDriUijOXuZRvwUdo...,tvlrau38-rJgb-ggv6-rzxc-v3ngebv0e00d,Concept Squad,37:0021crw9eeo2v477n5bcyhdkf9zemolgz63z4jck0ydb1,Training,"assigned to <a href=\n""http://support.appceler...",[huey.dunbar@example.com],2018-08-14T00:00:00Z,email-message
10,"['robert.langdon@example.com', 'peter.st@examp...",AAMKAYIJtPMTNywhyyXdWPCcTWjPPClYHaDonwloarWLnM...,60659,2018-08-14T00:00:00Z,60659,AAMKAYIJtPMTNywhyyXdWPCcTWjPPClYHaDonwloarWLnM...,dcdtozsa-r4dh-3buz-j7qt-onvge5qxco8f,Hexspeak,37:0021crw9eeo2v477n5bcyhdkf9zemolgz63z4jck0ydb1,Training,description\nthe purpose of this services is t...,[robert.langdon@example.com],2018-08-14T00:00:00Z,email-message
15,"['oliver.wilson@example.com', 'timothy.price@e...",AAMKAIidqMCjMLkQBcsfHCOjOOKIqSOwXJhBJBDqMUtVcP...,60677,2018-08-14T00:00:00Z,60677,AAMKAIidqMCjMLkQBcsfHCOjOOKIqSOwXJhBJBDqMUtVcP...,dcdtozsa-r4dh-3buz-j7qt-onvge5qxco8f,Hexspeak,37:0021crw9eeo2v477n5bcyhdkf9zemolgz63z4jck0ydb1,Training,my tiapp.xml file contains about 7 windows. al...,[peter.miller@gmail.com],2018-08-14T00:00:00Z,email-message
20,"['daniel.kopans@example.com', 'tommy.ivo@examp...",AAMKArAFMDpNrkxnVzCeSVqmLjoRhIIfnoPKizcEDVufsu...,60757,2018-08-14T00:00:00Z,60757,AAMKArAFMDpNrkxnVzCeSVqmLjoRhIIfnoPKizcEDVufsu...,dcdtozsa-r4dh-3buz-j7qt-onvge5qxco8f,Hexspeak,37:0021crw9eeo2v477n5bcyhdkf9zemolgz63z4jck0ydb1,Training,i've been tinkering around with ti to see if i...,[denis.ducharme@gmail.com],2018-08-14T00:00:00Z,email-message


In [88]:
main_df['from'] = main_df['from'].apply(lambda x: x[0])
main_df['from'] = main_df['from'].apply(user_dict_c)


In [89]:
main_df['from'].loc[0]

[{'user_id': 'christopher.franke@gmail.com',
  'user_display_name': 'Christopher Franke'}]

In [90]:
# main_df.reset_index(inplace=True, drop=True)

In [91]:
main_df['message_id'] = main_df['conversation_id']
main_df['id'] = main_df['conversation_id']

In [92]:
def add_permissions(df):
    add_p = []
    for i in df:
        add_p.append(i)
        if i in fake_tr:
            add_p.append(cog_dict[i])
    return add_p
main_df['_allow_permissions'] = main_df['_allow_permissions'].apply(evaluator)
main_df['_allow_permissions'] = main_df['_allow_permissions'].apply(add_permissions)

In [93]:
main_df.drop(columns=['type'], inplace=True)

In [94]:
main_df['message_type'] = "MESSAGE"

In [95]:
main_df.head()

,_allow_permissions,id,conversation_id,date,conversation_id,message_id,team_id,team_name,channel_id,channel_name,body,from,date,message_type
0,"[manojpraveen445@gmail.com, USER_1f4c0330-32c9...",60016,60016,2018-08-14T00:00:00Z,60016,60016,dcdtozsa-r4dh-3buz-j7qt-onvge5qxco8f,Hexspeak,37:0021crw9eeo2v477n5bcyhdkf9zemolgz63z4jck0ydb1,Training,just thought i'd let you know that in titanium...,"[{'user_id': 'christopher.franke@gmail.com', '...",2018-08-14T00:00:00Z,MESSAGE
5,"[cyndy.violette@example.com, USER_3e7bca78-5ee...",60648,60648,2018-08-14T00:00:00Z,60648,60648,tvlrau38-rJgb-ggv6-rzxc-v3ngebv0e00d,Concept Squad,37:0021crw9eeo2v477n5bcyhdkf9zemolgz63z4jck0ydb1,Training,"assigned to <a href=\n""http://support.appceler...",[{'user_id': 'USER_b570965f-414c-4868-9dc2-ae2...,2018-08-14T00:00:00Z,MESSAGE
10,"[robert.langdon@example.com, USER_e0f78954-fab...",60659,60659,2018-08-14T00:00:00Z,60659,60659,dcdtozsa-r4dh-3buz-j7qt-onvge5qxco8f,Hexspeak,37:0021crw9eeo2v477n5bcyhdkf9zemolgz63z4jck0ydb1,Training,description\nthe purpose of this services is t...,[{'user_id': 'USER_e0f78954-fabb-4faf-852e-de8...,2018-08-14T00:00:00Z,MESSAGE
15,"[oliver.wilson@example.com, USER_575901f1-ff79...",60677,60677,2018-08-14T00:00:00Z,60677,60677,dcdtozsa-r4dh-3buz-j7qt-onvge5qxco8f,Hexspeak,37:0021crw9eeo2v477n5bcyhdkf9zemolgz63z4jck0ydb1,Training,my tiapp.xml file contains about 7 windows. al...,"[{'user_id': 'peter.miller@gmail.com', 'user_d...",2018-08-14T00:00:00Z,MESSAGE
20,"[daniel.kopans@example.com, USER_96b3cc5a-b028...",60757,60757,2018-08-14T00:00:00Z,60757,60757,dcdtozsa-r4dh-3buz-j7qt-onvge5qxco8f,Hexspeak,37:0021crw9eeo2v477n5bcyhdkf9zemolgz63z4jck0ydb1,Training,i've been tinkering around with ti to see if i...,"[{'user_id': 'denis.ducharme@gmail.com', 'user...",2018-08-14T00:00:00Z,MESSAGE


In [96]:
rec_df = main_df.to_dict('records')

C:\Users\manoj praveen\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  """Entry point for launching an IPython kernel.


In [97]:
rec_df[0]

{'_allow_permissions': ['manojpraveen445@gmail.com',
  'USER_1f4c0330-32c9-49dd-8e25-4fc6347f9703',
  'christopher.franke@gmail.com',
  'richard.ellis@example.com',
  'USER_374b2ff7-020f-4571-959c-b0a3826702d7',
  'liam.aylward@example.com',
  'USER_d86f7831-4e24-4744-9c2f-325c2f1099ea',
  'lu.shanglei@gmail.com',
  'laurent.beaudoin@gmail.com',
  'rhonda.glenn@example.com',
  'USER_8dd3acad-f6aa-4f69-808b-a35a4a3aeec5'],
 'id': 60016,
 'conversation_id': 60016,
 'date': '2018-08-14T00:00:00Z',
 'message_id': 60016,
 'team_id': 'dcdtozsa-r4dh-3buz-j7qt-onvge5qxco8f',
 'team_name': 'Hexspeak',
 'channel_id': '37:0021crw9eeo2v477n5bcyhdkf9zemolgz63z4jck0ydb1',
 'channel_name': 'Training',
 'body': 'just thought i\'d let you know that in titanium technologies 1.2.2 i\nam now getting the following errors when i launch an app in the systematic\nvariables simulator:\n\n[info] one moment, termsing ...\n[info] detected third-party module: jp.masuidrive.ti.admob/0.1\n[info] titanium sdk version

In [98]:
len(channel_names)

6

In [99]:
messages = main_df[['conversation_id', 'message_id', 'team_id', 'team_name', 'channel_id',
       'channel_name', 'body', 'from', 'date', 
          'message_type' ]].copy()

In [100]:
messages = json.loads(json.dumps(messages.to_dict('records')))

C:\Users\manoj praveen\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  """Entry point for launching an IPython kernel.


In [101]:
len(messages), len(main_df)

(1936, 1936)

In [102]:
main_df["message"] = messages

In [103]:
main_df.columns

Index(['_allow_permissions', 'id', 'conversation_id', 'date',
       'conversation_id', 'message_id', 'team_id', 'team_name', 'channel_id',
       'channel_name', 'body', 'from', 'date', 'message_type', 'message'],
      dtype='object')

In [104]:
main_df = main_df[['_allow_permissions', 'id', 'conversation_id', 'date', 'message']].copy()

In [105]:
main_df = main_df.loc[:,~main_df.columns.duplicated()]

In [106]:
for i in main_df:
    if type(main_df[i].loc[0]) == np.int64:
        main_df[i] = main_df[i].astype(str)

In [107]:
rec_df = main_df.to_dict('records')

In [108]:
main_df.columns

Index(['_allow_permissions', 'id', 'conversation_id', 'date', 'message'], dtype='object')

In [109]:
main_df.index

Int64Index([   0,    5,   10,   15,   20,   25,   30,   35,   40,   45,
            ...
            9627, 9632, 9637, 9642, 9647, 9652, 9657, 9662, 9667, 9672],
           dtype='int64', length=1936)

In [110]:
replies_df = df.drop(labels=list(main_df.index))

In [111]:
replies_df = replies_df[['conversationId ', 'id ',
                   'team_id', 'teams', 'channel_id',
       'channel_name', 'comment', 'commentBy', 'commentDate', 'mentions']].copy()

In [112]:
replies_df.columns = ['conversation_id', 'id',
                    'team_id', 'team_name', 'channel_id',
       'channel_name', 'body', 'from', 'date', 'mentions']

In [113]:
replies_df['message_type'] = "REPLY"

In [114]:
replies_df['from'] = replies_df['from'].apply(user_dict_c)

In [115]:
replies_df['mentions'].fillna("NAN", inplace = True)

In [116]:
replies_df[replies_df['conversation_id'] == '60016']

,conversation_id,id,team_id,team_name,channel_id,channel_name,body,from,date,mentions,message_type


In [117]:
r_df = replies_df[replies_df['conversation_id'] == 60016]

In [118]:
r_df

,conversation_id,id,team_id,team_name,channel_id,channel_name,body,from,date,mentions,message_type
1,60016,AAMKApbePeTmSuCChwhQUENHHXNbetmzPEaiYANaDElQeL...,dcdtozsa-r4dh-3buz-j7qt-onvge5qxco8f,Hexspeak,37:0021crw9eeo2v477n5bcyhdkf9zemolgz63z4jck0ydb1,Training,"michael,\n\nif you are willing to hack up your...","[{'user_id': 'laurent.beaudoin@gmail.com', 'us...",2018-08-14T18:16:00Z,,REPLY
2,60016,AAMKADdThFgcKsogapXsgTaLemanknIISsXqsfIrRIJIGf...,dcdtozsa-r4dh-3buz-j7qt-onvge5qxco8f,Hexspeak,37:0021crw9eeo2v477n5bcyhdkf9zemolgz63z4jck0ydb1,Training,i'm experiencing this with 2.1.3rc,"[{'user_id': 'lu.shanglei@gmail.com', 'user_di...",2018-08-14T21:37:00Z,,REPLY
3,60016,AAMKAIVHcvnXXiVcjQxbKxpcJaatXnNQbbWwQeXkVjjzbP...,dcdtozsa-r4dh-3buz-j7qt-onvge5qxco8f,Hexspeak,37:0021crw9eeo2v477n5bcyhdkf9zemolgz63z4jck0ydb1,Training,pr pending personal://github.com/appcelerator/...,[{'user_id': 'USER_8dd3acad-f6aa-4f69-808b-a35...,2018-08-14T07:17:00Z,[Lu Shanglei],REPLY
4,60016,AAMKACGbgoQFaeYHAANbDTLbkAoEqOsLbnjedXBYCHprEm...,dcdtozsa-r4dh-3buz-j7qt-onvge5qxco8f,Hexspeak,37:0021crw9eeo2v477n5bcyhdkf9zemolgz63z4jck0ydb1,Training,http://technologies.appcelerator.com/question/...,[{'user_id': 'USER_1f4c0330-32c9-49dd-8e25-4fc...,2018-08-14T07:15:00Z,,REPLY


In [119]:
rec_ddf = replies_df.to_dict('records')

In [120]:
rec_ddf

[{'conversation_id': 60016,
  'id': 'AAMKApbePeTmSuCChwhQUENHHXNbetmzPEaiYANaDElQeLWupOFKkZxCeopUQFmHIqvEsFnBPTFgeJbdyQfKsIUDViquGWHsDFhqsdtxAYePVZKIpvHLyUckvTSpAhKxOpfZcbJysQTBquvNUafYMyWpmbVLG=',
  'team_id': 'dcdtozsa-r4dh-3buz-j7qt-onvge5qxco8f',
  'team_name': 'Hexspeak',
  'channel_id': '37:0021crw9eeo2v477n5bcyhdkf9zemolgz63z4jck0ydb1',
  'channel_name': 'Training',
  'body': 'michael,\n\nif you are willing to hack up your iphone/termser.py file the fix is really simple, two back slashes to escape spaces in paths:\n\npersonal://github.com/appcelerator/titanium_mobile/pull/2866/files',
  'from': [{'user_id': 'laurent.beaudoin@gmail.com',
    'user_display_name': 'Laurent Beaudoin'}],
  'date': '2018-08-14T18:16:00Z',
  'mentions': '',
  'message_type': 'REPLY'},
 {'conversation_id': 60016,
  'id': 'AAMKADdThFgcKsogapXsgTaLemanknIISsXqsfIrRIJIGfuvKrDJPBcfASQMlnVtIEWbXOlydZTUuteUEBTvvRwXDaxkMrnvkJvDlcaAbQgWQMJdySiRhNBIFaZEzRNLRkIvdhagLmKLRZIaroEfEulVJngLT=',
  'team_id': 'dcdtozsa-

In [121]:
type(r_df['conversation_id'].loc[1])

numpy.int64

In [122]:
for i in replies_df:
    if type(replies_df[i].loc[1]) == np.int64:
        replies_df[i] = replies_df[i].astype(str)

In [123]:
replies_df.head()

,conversation_id,id,team_id,team_name,channel_id,channel_name,body,from,date,mentions,message_type
1,60016,AAMKApbePeTmSuCChwhQUENHHXNbetmzPEaiYANaDElQeL...,dcdtozsa-r4dh-3buz-j7qt-onvge5qxco8f,Hexspeak,37:0021crw9eeo2v477n5bcyhdkf9zemolgz63z4jck0ydb1,Training,"michael,\n\nif you are willing to hack up your...","[{'user_id': 'laurent.beaudoin@gmail.com', 'us...",2018-08-14T18:16:00Z,,REPLY
2,60016,AAMKADdThFgcKsogapXsgTaLemanknIISsXqsfIrRIJIGf...,dcdtozsa-r4dh-3buz-j7qt-onvge5qxco8f,Hexspeak,37:0021crw9eeo2v477n5bcyhdkf9zemolgz63z4jck0ydb1,Training,i'm experiencing this with 2.1.3rc,"[{'user_id': 'lu.shanglei@gmail.com', 'user_di...",2018-08-14T21:37:00Z,,REPLY
3,60016,AAMKAIVHcvnXXiVcjQxbKxpcJaatXnNQbbWwQeXkVjjzbP...,dcdtozsa-r4dh-3buz-j7qt-onvge5qxco8f,Hexspeak,37:0021crw9eeo2v477n5bcyhdkf9zemolgz63z4jck0ydb1,Training,pr pending personal://github.com/appcelerator/...,[{'user_id': 'USER_8dd3acad-f6aa-4f69-808b-a35...,2018-08-14T07:17:00Z,[Lu Shanglei],REPLY
4,60016,AAMKACGbgoQFaeYHAANbDTLbkAoEqOsLbnjedXBYCHprEm...,dcdtozsa-r4dh-3buz-j7qt-onvge5qxco8f,Hexspeak,37:0021crw9eeo2v477n5bcyhdkf9zemolgz63z4jck0ydb1,Training,http://technologies.appcelerator.com/question/...,[{'user_id': 'USER_1f4c0330-32c9-49dd-8e25-4fc...,2018-08-14T07:15:00Z,,REPLY
6,60648,AAMKAouEZTynBkdLvvCIxZlzuSbezdCBBCajeTilnDrMTY...,tvlrau38-rJgb-ggv6-rzxc-v3ngebv0e00d,Concept Squad,37:0021crw9eeo2v477n5bcyhdkf9zemolgz63z4jck0ydb1,Training,i've tracked this down (on window's xp) to a f...,[{'user_id': 'USER_96ec5a12-8d0d-4c7f-b62c-215...,2018-08-14T19:22:00Z,,REPLY


In [124]:
def convert(o):
    if isinstance(o, np.generic): return o.item()  
    raise TypeError

json.dumps({'value': np.int64(r_df['conversation_id'].loc[1])}, default=convert)

'{"value": 60016}'

In [125]:
ys = json.dumps(rec_ddf[0]) + ',' + json.dumps(rec_ddf[1])

In [126]:
ys

'{"conversation_id": 60016, "id": "AAMKApbePeTmSuCChwhQUENHHXNbetmzPEaiYANaDElQeLWupOFKkZxCeopUQFmHIqvEsFnBPTFgeJbdyQfKsIUDViquGWHsDFhqsdtxAYePVZKIpvHLyUckvTSpAhKxOpfZcbJysQTBquvNUafYMyWpmbVLG=", "team_id": "dcdtozsa-r4dh-3buz-j7qt-onvge5qxco8f", "team_name": "Hexspeak", "channel_id": "37:0021crw9eeo2v477n5bcyhdkf9zemolgz63z4jck0ydb1", "channel_name": "Training", "body": "michael,\\n\\nif you are willing to hack up your iphone/termser.py file the fix is really simple, two back slashes to escape spaces in paths:\\n\\npersonal://github.com/appcelerator/titanium_mobile/pull/2866/files", "from": [{"user_id": "laurent.beaudoin@gmail.com", "user_display_name": "Laurent Beaudoin"}], "date": "2018-08-14T18:16:00Z", "mentions": "", "message_type": "REPLY"},{"conversation_id": 60016, "id": "AAMKADdThFgcKsogapXsgTaLemanknIISsXqsfIrRIJIGfuvKrDJPBcfASQMlnVtIEWbXOlydZTUuteUEBTvvRwXDaxkMrnvkJvDlcaAbQgWQMJdySiRhNBIFaZEzRNLRkIvdhagLmKLRZIaroEfEulVJngLT=", "team_id": "dcdtozsa-r4dh-3buz-j7qt-onvge5qxco8

In [127]:
json.dumps(eval(ys))

'[{"conversation_id": 60016, "id": "AAMKApbePeTmSuCChwhQUENHHXNbetmzPEaiYANaDElQeLWupOFKkZxCeopUQFmHIqvEsFnBPTFgeJbdyQfKsIUDViquGWHsDFhqsdtxAYePVZKIpvHLyUckvTSpAhKxOpfZcbJysQTBquvNUafYMyWpmbVLG=", "team_id": "dcdtozsa-r4dh-3buz-j7qt-onvge5qxco8f", "team_name": "Hexspeak", "channel_id": "37:0021crw9eeo2v477n5bcyhdkf9zemolgz63z4jck0ydb1", "channel_name": "Training", "body": "michael,\\n\\nif you are willing to hack up your iphone/termser.py file the fix is really simple, two back slashes to escape spaces in paths:\\n\\npersonal://github.com/appcelerator/titanium_mobile/pull/2866/files", "from": [{"user_id": "laurent.beaudoin@gmail.com", "user_display_name": "Laurent Beaudoin"}], "date": "2018-08-14T18:16:00Z", "mentions": "", "message_type": "REPLY"}, {"conversation_id": 60016, "id": "AAMKADdThFgcKsogapXsgTaLemanknIISsXqsfIrRIJIGfuvKrDJPBcfASQMlnVtIEWbXOlydZTUuteUEBTvvRwXDaxkMrnvkJvDlcaAbQgWQMJdySiRhNBIFaZEzRNLRkIvdhagLmKLRZIaroEfEulVJngLT=", "team_id": "dcdtozsa-r4dh-3buz-j7qt-onvge5qxc

In [128]:
for i in replies_df[replies_df['conversation_id'] == 60648]:
    print(i)

conversation_id
id
team_id
team_name
channel_id
channel_name
body
from
date
mentions
message_type


In [129]:
# d1 = replies_df[replies_df['conversation_id'] == 60016].iloc[1].to_dict()
# d2 = replies_df[replies_df['conversation_id'] == 60016].iloc[2].to_dict()

In [130]:
# d1, d2

In [131]:
# from itertools import chain
# dict(chain.from_iterable(d.items() for d in (d1, d2)))

In [132]:
rec_df[0]

{'_allow_permissions': ['manojpraveen445@gmail.com',
  'USER_1f4c0330-32c9-49dd-8e25-4fc6347f9703',
  'christopher.franke@gmail.com',
  'richard.ellis@example.com',
  'USER_374b2ff7-020f-4571-959c-b0a3826702d7',
  'liam.aylward@example.com',
  'USER_d86f7831-4e24-4744-9c2f-325c2f1099ea',
  'lu.shanglei@gmail.com',
  'laurent.beaudoin@gmail.com',
  'rhonda.glenn@example.com',
  'USER_8dd3acad-f6aa-4f69-808b-a35a4a3aeec5'],
 'id': '60016',
 'conversation_id': '60016',
 'date': '2018-08-14T00:00:00Z',
 'message': {'conversation_id': 60016,
  'message_id': 60016,
  'team_id': 'dcdtozsa-r4dh-3buz-j7qt-onvge5qxco8f',
  'team_name': 'Hexspeak',
  'channel_id': '37:0021crw9eeo2v477n5bcyhdkf9zemolgz63z4jck0ydb1',
  'channel_name': 'Training',
  'body': 'just thought i\'d let you know that in titanium technologies 1.2.2 i\nam now getting the following errors when i launch an app in the systematic\nvariables simulator:\n\n[info] one moment, termsing ...\n[info] detected third-party module: jp.mas

In [133]:
s_df = ''
for j in range(len(replies_df[replies_df['conversation_id'] == '60016'])):
    s_df = s_df + json.dumps(replies_df[replies_df['conversation_id'] == '60016'].iloc[j].to_dict()) + ', '

In [134]:
main_df.head()

,_allow_permissions,id,conversation_id,date,message
0,"[manojpraveen445@gmail.com, USER_1f4c0330-32c9...",60016,60016,2018-08-14T00:00:00Z,"{'conversation_id': 60016, 'message_id': 60016..."
5,"[cyndy.violette@example.com, USER_3e7bca78-5ee...",60648,60648,2018-08-14T00:00:00Z,"{'conversation_id': 60648, 'message_id': 60648..."
10,"[robert.langdon@example.com, USER_e0f78954-fab...",60659,60659,2018-08-14T00:00:00Z,"{'conversation_id': 60659, 'message_id': 60659..."
15,"[oliver.wilson@example.com, USER_575901f1-ff79...",60677,60677,2018-08-14T00:00:00Z,"{'conversation_id': 60677, 'message_id': 60677..."
20,"[daniel.kopans@example.com, USER_96b3cc5a-b028...",60757,60757,2018-08-14T00:00:00Z,"{'conversation_id': 60757, 'message_id': 60757..."


In [135]:
main_df['conversation_id'].loc[0]

'60016'

In [136]:
lk = iter(rec_df)

In [137]:
final_res = []
for i in main_df['conversation_id']:
    s_df = ''
    for j in range(len(replies_df[replies_df['conversation_id'] == i])):
        s_df = s_df + json.dumps(replies_df[replies_df['conversation_id'] == i].iloc[j].to_dict()) + ', '
    sub_dict = {"replies": list(eval(s_df[:-2]))}
    fin = json.dumps(dict(list(next(lk).items()) + list(sub_dict.items())))
    final_res.append(json.loads(fin))

In [138]:
final_res[0]

{'_allow_permissions': ['manojpraveen445@gmail.com',
  'USER_1f4c0330-32c9-49dd-8e25-4fc6347f9703',
  'christopher.franke@gmail.com',
  'richard.ellis@example.com',
  'USER_374b2ff7-020f-4571-959c-b0a3826702d7',
  'liam.aylward@example.com',
  'USER_d86f7831-4e24-4744-9c2f-325c2f1099ea',
  'lu.shanglei@gmail.com',
  'laurent.beaudoin@gmail.com',
  'rhonda.glenn@example.com',
  'USER_8dd3acad-f6aa-4f69-808b-a35a4a3aeec5'],
 'id': '60016',
 'conversation_id': '60016',
 'date': '2018-08-14T00:00:00Z',
 'message': {'conversation_id': 60016,
  'message_id': 60016,
  'team_id': 'dcdtozsa-r4dh-3buz-j7qt-onvge5qxco8f',
  'team_name': 'Hexspeak',
  'channel_id': '37:0021crw9eeo2v477n5bcyhdkf9zemolgz63z4jck0ydb1',
  'channel_name': 'Training',
  'body': 'just thought i\'d let you know that in titanium technologies 1.2.2 i\nam now getting the following errors when i launch an app in the systematic\nvariables simulator:\n\n[info] one moment, termsing ...\n[info] detected third-party module: jp.mas

In [147]:
with open("ENG_teams_chat.json", "w") as f:
    json.dump(final_res, f, sort_keys = False, indent = 4)
#     f.write(json.dumps(rec_df))
f.close()

In [139]:
# sub_dict = {"replies": list(eval(s_df[:-2]))}

In [140]:
# list(eval(s_df[:-2]))

In [141]:
# list(sub_dict.items())

In [142]:
# fin = json.dumps(rec_df[0]) + ', '+ json.dumps(sub_dict)
# fin = json.dumps(dict(list(rec_df[0].items()) + list(sub_dict.items())))

In [143]:
# eval(fin)

In [144]:
# dict(list(eval(fin)[0].items()) + list(eval(fin)[1].items()))